In [1]:
# import unittest
# from unittest import TestCase

import random
import syft as sy
from syft.core import utils
import torch
import torch.nn.functional as F
from torch.autograd import Variable as Var
import json

hook = sy.TorchHook(verbose=True)

me = hook.local_worker
me.is_client_worker = False

bob = sy.VirtualWorker(id="bob", hook=hook, is_client_worker=False)
alice = sy.VirtualWorker(id="alice", hook=hook, is_client_worker=False)
james = sy.VirtualWorker(id="james", hook=hook, is_client_worker=False)

me.add_workers([bob, alice, james])
bob.add_workers([me, alice, james])
alice.add_workers([me, bob, james])
james.add_workers([me, bob, alice])

param = []

In [2]:
torch.manual_seed(42)

# create our dataset
data = Var(torch.FloatTensor([[0, 0], [0, 1], [1, 0], [1, 1]]))
target = Var(torch.FloatTensor([[0], [0], [1], [1]]))

data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

data_alice = data[2:].send(alice)
target_alice = target[2:].send(alice)

# create our model
model = torch.nn.Linear(2, 1)

opt = torch.optim.SGD(params=model.parameters(), lr=0.1)

datasets = [(data_bob, target_bob), (data_alice, target_alice)]


for iter in range(2):

    for data, target in datasets:
        model.send(data.location)

        # update the model
        model.zero_grad()
        pred = model(data)
        loss = ((pred - target)**2).sum()
        loss.backward()
        opt.step()

        model.get()
        if(iter == 1):
            final_loss = loss.get().data[0]

assert final_loss == 0.18085284531116486

0
asdf
asdf
1
asdf
asdf


In [20]:
data_bob.location

<syft.core.workers.virtual.VirtualWorker id:bob>

In [21]:
model.send(bob)

In [24]:
pred = model(data_bob)

0
asdf


KeyError: 26877416937

In [3]:
x

[0.5406103730201721,
 0.5869042277336121,
 -0.16565567255020142,
 0.6732476353645325,
 0.5102827548980713,
 -0.0841369703412056]

In [3]:
data.send(bob)

Variable containing:FloatTensor[_PointerTensor - id:6761607226 owner:0 loc:bob id@loc:976960384]

In [4]:
bob._objects[data.grad.id_at_location].data.id

6748238397

In [5]:
bob._objects

{976960384: [_LocalTensor - id:976960384 owner:bob],
 6748238397: [_LocalTensor - id:6748238397 owner:bob],
 71210702916: [_LocalTensor - id:71210702916 owner:bob],
 73189406364: [_LocalTensor - id:73189406364 owner:bob]}

KeyError: 46847008124

NotImplementedError: ('All arguments should share the same child type.', [<class 'syft.core.frameworks.torch.tensor._LocalTensor'>, <class 'syft.core.frameworks.torch.tensor._PointerTensor'>])

NotImplementedError: ('All arguments should share the same child type.', [<class 'syft.core.frameworks.torch.tensor._LocalTensor'>, <class 'syft.core.frameworks.torch.tensor._PointerTensor'>])